In [ ]:
#%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime

import createFolder as cF
import calc_date as cd
import plot_sfc_spaghetti as spagh
import save_fig as SF
import get_Haukeli_obs_data as obsDat
import calc_48h_acc as acc
import fill_values as fv

import os

import pandas as pd

In [ ]:
year = 2016
month = 12
#tid = '00'
tid = '18'
station = 'Haukeliseter'
savefig = 0  # 1 = yes, 0 = no

### if plot with double fence data
dofe = 1     # 1 = yes, 0 = no

In [ ]:
#### days 
t = []
fi = np.arange(1,16)
sec00 = np.arange(17,30)
sec18 = np.arange(17,29)

t = np.concatenate((t,fi))

if tid == '00':    
    t = np.concatenate((t,sec00))
elif tid == '18':
    t = np.concatenate((t,sec18))


In [ ]:
level = 'sfc'

ncdir = '../../MEPS_data/%s/%s_%s' %(station,level,tid)
txtdir = '../../eklima/'
txt_filename = 'Observations-hour_minutes.txt'

var_name = 'precipitation_amount_acc'
unit = '[mm]'
figdir = '../../MEPS_fig/%s' %(var_name)
figdir2 = '../../MEPS_fig/%s/Morten' %(var_name)
#var_name = 'liquid_water_content_of_surface_snow'
# Folder where the figures are being saved    
cF.createFolder('%s/' %(figdir))
cF.createFolder('%s/' %(figdir2))
form = 'png'


In [ ]:
# read in the Haukeliseter observation file, sorted by daily values
Haukeli = pd.read_csv('%s/%s' %(txtdir, txt_filename),\
                    sep = ';',header=19, skipfooter = 11, engine= 'python')

DateHour = Haukeli['Date-Hour(UTC)']

RR_1 = Haukeli['RR_1'].astype(float)           # Precipitation (1 hour); [mm]

if tid == '18':
    precip, dt_precip, hour_precip, day_precip, month_precip, year_precip = obsDat.valid_values_18UTC(RR_1, DateHour)
    dt, p, hour, dy, mm, yr = obsDat.arange_daily_18UTC(RR_1, DateHour)
if tid == '00':
    precip, dt_precip, hour_precip, day_precip, month_precip, year_precip = obsDat.valid_values(RR_1, DateHour)
    dt, p, hour, dy, mm, yr = obsDat.arange_daily(RR_1, DateHour)

   

In [ ]:
fn_sfc = dict()
time_sfc = dict()
ini_day = []
#hh = []
calday = []
calmon = []
precipitation_amount_acc = dict()
lead_time_sfc = dict()
title = dict()


for day in t:
    if day < 10:
        day = '0%s' %(int(day))
    else:
        day = '%s' %(int(day))
    var_name_sfc = '%s' %(var_name)
    
    for ens_memb in range(0,10):
# read in the netcdf surface file
#        fn_sfc[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir,year,month,day,tid,ens_memb))
        fn_sfc[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir,year,month,day,tid,ens_memb))
        time_sfc[ens_memb] = fn_sfc[ens_memb].variables['time']
        time_sfc[ens_memb] = fv.fill_nan(time_sfc[ens_memb][:])
# initialisation time for plotting
        ini_day = (datetime.datetime.utcfromtimestamp(time_sfc[0][0]).day)   # day of initialisation
        hh = (datetime.datetime.utcfromtimestamp(time_sfc[0][0]).hour)       # first hour of initialisation
        calday, calmon = cd.get_dayname(year, month, ini_day)
# Read in the variable name which should be plotted  
        precipitation_amount_acc[ens_memb] = fn_sfc[ens_memb].variables[var_name]



# lead time
        lead_time_sfc[ens_memb] = np.arange(0,time_sfc[ens_memb].shape[0])
# get only valuable values not nan
        #substitute missing values with nan 
        precipitation_amount_acc[ens_memb] = fv.fill_nan(precipitation_amount_acc[ens_memb][:,:])
# get only valuable values not nan
        time_sfc[ens_memb] = time_sfc[ens_memb][~np.isnan(time_sfc[ens_memb])]
        lead_time_sfc[ens_memb] = lead_time_sfc[ens_memb][~np.isnan(precipitation_amount_acc[ens_memb]).any(axis=1)]
        precipitation_amount_acc[ens_memb] = precipitation_amount_acc[ens_memb][~np.isnan(precipitation_amount_acc[ens_memb]).any(axis=1),:]
    
#### calculate every 60 min the accumulation over 48 hours
    tot = acc.accumulation_dt60_for48h(precip, ini_day)

    

# spaghetti plot with all ensemble member + control
    if ini_day < 10:
        ini_day = '0%s' %(ini_day)
    else:
        ini_day = '%s' %(ini_day)
    
    if hh < 10:
        hh = '0%s' %(hh)
    else:
        hh = '%s' %(hh)
        
    fig_name = '%s%s%s_%s.png' %(year,month,ini_day,hh)
    title = 'initalised: %s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh)
    var_name_sfc = var_name_sfc.split("_")
    
    if len(tot) <= 3:
        print('no double fence data')
        dofe = 0
        Xmax = 49
    else:
        dofe = 1
        Xmax = np.asarray(tot).shape[0]
    spagh.spaghetti_sfc_dofe(lead_time_sfc, precipitation_amount_acc, tot, Xmax, day, var_name_sfc,
                        hour, mm, dy, yr, int(ini_day), unit, title, tid, dofe)
#    Xmax = np.asarray(tot_60)[:,(int(day)-1)].shape[0]
#    spagh.spaghetti_sfc_dofe(lead_time_sfc, precipitation_amount_acc, precip, time_sfc[0], Xmax, 
 #                               day, var_name_sfc, unit, title, tid, dofe)
    if savefig == 1:
        SF.save_figure_landscape(figdir, fig_name, form)
        print('plot saved: %s/%s' %(figdir,fig_name))
    elif safefig == 0:
        plt.show()
    plt.close()
    
    if tid == '18':
        Xmax = 31
        title = 'Initialised %s%s%sT%sZ' %(year, month, ini_day,hh)
        spagh.spaghetti_sfc_dofe_Morten(lead_time_sfc, precipitation_amount_acc, tot, Xmax, 
                                day, var_name_sfc, unit, title, tid)
        if savefig == 1:
            SF.save_figure_landscape(figdir2, fig_name, form)
            print('plot saved: %s/%s' %(figdir2,fig_name))
        elif savefig == 0:
            plt.show()
        plt.close()


        
    for ens_memb in range(0,10):
        fn_sfc[ens_memb].close()
    